## 문서 분류 모델 훑어보기
- 문서 분류란 문서가 주어졌을 때 해당 문서의 범주를 분류하는 과제이다.
- 뉴스를 입력하고 정치, 경제, 연예 등 범주를 맞히거나, 영화 리뷰가 긍정/부정 등 어떤 극성을 가지는지 분류

- 이번에 사용할 데이터는 네이버 영화 리뷰 말뭉치(NSMC)
- 문서 분류 모델은 영화 리뷰 문장을 입력으로 하고 해당 문장이 속한 극성의 확률을 출력한다.
- 문서 분류 모델의 출력은 확률값이므로 적당한 후처리 과정을 거쳐 긍정, 부정처럼 사람이 보기에 좋은 형태로 가공해준다.

### 모델 구조
- 여기서 사용하는 문서분류 모델은 입력 문장을 토큰화한 뒤 
- 문장 시작과 끝을 알리는 스페셜 토큰 CLS와 SEP를 각각 원래 토큰 시퀀스 앞뒤에 붙인다. 
- 이를 BERT 모델에 입력하고 문장 수준의 벡터(pooler_output)를 뽑는다.
- 이 벡터에 작은 추가 모듈을 덧붙여 모델 전체의 출력이 [해당 문장이 긍정일 확률, 해당 문장이 부정일 확률] 형태가 되도록한다.


### 1. 모델 환경 설정
- kcbert-base 모델을 NSMC 데이터로 파인튜닝해보자

In [1]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="nlpbook/checkpoint-doccls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

TrainArguments의 매개변수
- pretrained_model_name: 프리트레인 마친 언어 모델의 이름(단 해당 모델은 허깅페이스 모델 허브에 등록되어 있어야함)
- downstream_corpus_name: 다운스트림 데이터의 이름
- downstream_model_root_dir: 다운스트림 데이터를 내려받을 위치. 입력하지 않으면 /content/Korpora에 저장됨
- downstream_model_dir: 파인튜닝된 모델의 체크포인트가 저장될 위치.
- batch_size: 배치 크기. 하드웨어 가속기로 GPU를 선택(torch.cuda.is_available()==True)했다면 32, TPU(torch.cuda.is_available()==False)라면 4/ 코랩환경에서 TPU는 보통 8개 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기이므로 이렇게 설정해둔다.
- learning_rate: 러닝 레이트(보폭). 1회 스텝에서 모델을 얼마나 업데이트할지에 관한 크기를 가리킨다.
- max_seq_length: 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 max_seq_length로 자르고 짧은 문장은 max_seq_length가 되도록 스페셜 토큰([PAD])를 붙여줌
- epochs: 학습 에포크 수. 3이라면 학습 데이터 전체를 3회 반복
- tpu_cores: TPU 코어수. 하드웨어 가속기로 GPU를 선택했다면 0,TPU라면 8
- seed: 랜덤 시드

In [2]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


- args에 지정된 시드로 고정하는 역할
- 같은 시드를 사용하면 컴퓨터는 계속 같은 패턴의 난수를 생성함

In [3]:
nlpbook.set_logger(args)

### 2. 말뭉치 내려받기
- NSMC 데이터를 내려받자.
- kopora라이브러리로 데이터를 내려받고 corpus_name(nsmc)에 해당하는 말뭉치르 root_dir아래에 저장

In [4]:
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=True,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 19.2MB/s]                                                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 18.6MB/s]                                                             


### 3. 토크나이저 준비하기
- 여기서 다루는 데이터의 기본단위는 텍스트 형태의 문장이다.
- 토큰화란 문장을 토큰 시퀀스로 분절하는 과정을 가리킨다.
- 여기서 사용하는 모델은 자연어 문장을 분절한 토큰 시퀀스를 입력으로 받는다.
- kcbert-base모델이 사용하는 토크나이저를 선언

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

### 4. 데이터 전처리하기
- 딥러닝 모델을 학습하려면 학습 데이터를 배치 단위로 계속 모델에 공급해 주어야 한다.
- 파이토치에선 이 역할을 데이터 로더가 수행한다.
- 데이터 로더는 데이터 셋이 보유하고 있는 인스턴스를 배치 크기만큼 뽑아 자료형, 데이터 길이 등 정해진 형식에 맞춰 배치를 만들어 준다.

In [8]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train"
)

ClassificationDataset 의 역할
- NsmcCorpus랑 tokenizer를 품고있음
- NsmcCorpus는 CSV 파일 형식의 NSMC 데이터를 문장(영화리뷰)과 레이블(긍정, 부정)로 읽어 들임
- NsmcCorpus는 ClassificationDataset이 요구하면 이 문장과 레이블을 ClassificationDataset에 제공함
- ClassificationDataset는 NsmcCorpus가 넘겨준 문장과 레이블 각각을 tokenizer를 활용해 모델이 학습 할 수 있는 형태(ClassificationFeatures)로 가공함
- ClassificationFeatures라는 자료형에는 4가지 정보가 있음
    - input_ids: 인덱스로 변환된 토큰 시퀀스(list(int))
    - attention_mask: 토큰(1)인지 패딩(0)인지(list(int))
    - token_type_ids: 세그먼트 정보(list(int))
    - label: 정수로 바뀐 레이블 정보(int)

In [9]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

- NsmcCorpus가 넘겨준 0번 데이터(text:아 더빙.. 진짜 짜증나네요 목소리, label: 0)가 ClassificationFeatures라는 0번 인스턴스로 변환된것
- train_dataset[0] 처럼 ClassificationDataset이 가지고 있는 모든 인스턴스는 인덱스로 접근할 수 있음
- train_dataset[0].input_ids, train_dataset[0].attention_mask, train_dataset[0].token_type_ids의 길이가 128인 이유는 토큰 기준 최대 길이(max_seq_length)를 코드의 args에서 128로 설정해 두었기 때문
- 세그먼트 정보(token_type_ids)를 입력하는 건 BERT 모델의 특징이다.
- BERT의 프리트레인 과제는 '빈칸 맞히기'외에 '이어진 문서인지 맞히기'도 있다.
- 문서 2개를 입력하고 2개의 문서가 이어진 것인지 아닌지를 이진 분류하는 과정에서 프리트레인을 수행한다
- BERT의 세그먼트 정보는 첫번째 문서에 해당하는 토큰 시퀀스가 0, 두번째 문서의 토큰 시퀀스가 1이 되도록 만든다.
- 하지만 우리는 영화리뷰 문서 하나를 입력하고 그 문서의 극성을 분류하는 과제를 수행 중이다.
- 따라서 이 실습에서 모든 인스턴스의 세그먼트 정보는 0으로 넣는다.

### 학습 데이터 로더 구축
- 데이터 로더는 ClassificationDataset 클래스가 들고 있는 전체 인스턴스 가운데 배치 크기(args의 batch_size)만큼을 뽑아 배치 형태로 가공(nlpbook.data_collator)하는 역할을 수행
- sampler: 샘플링 방식을 정의/ 여기서 데이터로더는 배치를 만들 때 ClassificationDataset이 들고 있는 전체 인스턴스 가운데 batch_size 개수만큼을 비복원(replacement=False) 랜덤추출(Random Sampler)한다.
- collate_fn: 이렇게 뽑은 인스턴스들을 배치로 만드는 역할/ nlpbook.data_collator는 같은 배치에서 인스턴스가 여럿일 때 이를 input_ids, attention_mask 등 종류별로 모으고 파이토치가 요구하는 자료형인 텐서 형태로 바꾸는 역할을 수행

In [10]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

### 테스트 데이터 로더 구축
- 평가용 데이터 로더는 SequentialSampler를 사용
- SequentialSampler는 인스턴스를 batch_size만큼 순서대로 추출하는 역할을 함
- 학습 때 배치 구성은 랜덤으로 하는것이 좋음
- 테스트할 때는 굳이 랜덤할 필요 없음

In [11]:
from torch.utils.data import SequentialSampler
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

### 5. 모델 불러오기
- pretrained_model_name을 beomi/kcbert-base로 지정했으므로 프리트레인을 마친 BERT로 kcbert-base를 사용
- 모델을 초기화하는 코드에서 BertForSequenceClassification은 프리트레인을 마친 BERT 모델 위에 문서 분류용 태스크 모듈이 덧붙여진 형태의 모델 클래스이다.
- 이 클래스는 허깅페이스에서 제공하는 transformers라이브러리에 포함돼있다.
- 허깅페이스 모델 허브에 등록된 모델이라면 별다른 코드 수정없이 kcbert-base 이외에 다른 언어 모델을 사용할 수 있다.
- 예를 들어 bert-base-uncased 모델은 구글이 공개한 다국어 BERT 모델인데 pretrained_model_name에 이 모델명을 입력하면 해당 모델을 쓸 수 있다. 허깅페이스에 등록된 모델 목록은 huggingface.co/models에서 확인할 수 있다.

In [12]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

### 6. 모델 학습시키기
- 파이토치 라이트닝이 제공하는 LightningModule 클래스를 상속받아 태스크를 정의한다.
- 태스크에는 모델과 옵티마이저, 학습과정 등이 정의돼 있다.
- 다음 코드를 실행하면 문서 분류용 태스크를 정의할 수 있다.
- model을 ClassificationTask에 주입한다.
- ClassificationTask에는 옵티마이저, 러닝 레이트 스케줄러가 정의돼있는데 옵티마이저로는 아담, 러닝 레이트 스케줄러로는 ExponentialLR을 사용한다.

In [13]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [ ]:
# trainer = nlpbook.get_trainer(args)
from pytorch_lightning import Trainer
    
trainer = Trainer()
trainer.fit(task, train_dataloader, val_dataloader)

c:\python39\lib\site-packages\pytorch_lightning\loops\utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: E:\workspace\ml\ML-100Days\nlp\lightning_logs
c:\python39\lib\site-packages\pytorch_lightning\core\optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.680   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

## 학습 마친 모델을 실전 투입하기


### 1. 인자값 설정

### 2. 토크나이저 및 모델 불러오기

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downsteam_model_dir="nlpbook/checkpoint-doccls",
    max_seq_length=128,
)

In [ ]:
# 토크나이저 로드
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [ ]:
# 체크포인트 로드
from torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_locatin=torch.device("cpu")
)

In [ ]:
# BERT 설정 로드
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt["state_dict"]["model.classifier.bias"].shape.numel(),
)

In [ ]:
# BERT 모델 초기화
from transformers import BertForSequenceClassification
model = BertForSequenceClassification(pretrained_model_config)

In [ ]:
# 체크 포인트 주입하기
model.load_state_dict({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt["state_dict"].items()})

In [ ]:
# 평가 모드로 전환
model.eval()

### 3. 모델 output만들고 후처리

In [ ]:
# 문장을 토큰화한 뒤 input_id, attention_masks, token_type_ids 만들기
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        # inputs를 파이토치 텐서로 바꾸기
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) # 모델 계산하기
        # 로짓에 소프트 맥스 취하기
        prob = outputs.logits.softmax(dim=1)
        # 긍정/부정 확률을 소수점 4자리로 반올림
        positive_prob = round(prob[0][1].item(),4)
        negative_prob = round(prob[0][0].item(),4)
        # 예측 확률의 최댓값 위치에 따라 pred 만들기
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
        return {
            'sentence': sentence,
            'prediction': pred,
            'positive_data': f'긍정 {positive_prob}',
            'negative_data': f'부정 {negative_prob}',
            'positive_width': f'긍정 {positive_prob * 100}%',
            'negative_width': f'부정 {negative_prob * 100}%',
            
        }

### 4. 웹 서비스 시작하기

In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

## Reference
- [영화리뷰 감정분석](https://www.youtube.com/watch?v=HY493c6n65E&list=PLIMb_GuNnFwdWabX_4pkti2cYonT-pFrw)
- [박은정 님의 네이버 영화평 말뭉치](https://github.com/e9t/nsmc)
- [NsmcCorpus, ClassificationDataset의 역할](https://ratsgo.github.io/nlpbook/docs/doc_cls/detail/)